In [99]:
# Source: Alexandru Tifrea and Fanny Yang, 2021.

# Python Notebook Commands
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from copy import deepcopy
import numpy as np
import time

import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio

import ipywidgets
from ipywidgets import interact, interactive, interact_manual

import sklearn
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold

from utils import generate_data, generate_additional_data, compute_population_risk, compute_empirical_risk, repeat_experiment

# Change these values if the images don't fit for your screen.
figure_width=1200
figure_height=500

colors = ["blue", "orange", "purple", "red"]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Choosing the number of CV folds - Trade-off

When choosing the number of cross-validation (CV) folds, we have to be aware of a trade-off that is illustrated in the figure below.

Let's assume that we perform $K$-fold CV and we use a data set $D$, with $n$ labeled samples. Let us denote by $D_T$ the $(K-1)$ splits that we use for training and by $D_V$ the split that we use for validation. We can choose $K$ such splits and we denote them as $D_T^{(k)}$ and $D_V^{(k)}$ for $k \in \{1, ..., K\}$. The cardinals of these sets are $\frac{n(K-1)}{K}$ and $\frac{n}{K}$ respectively.

On the one hand, we want to have enough samples in the training subset $D_T$ in order a good prediction function. If $K$ is too small, then $|D_T| << |D|$ and by training on $D_T$ we obtain an estimator that is much worse than what we could have gotten, had we trained on the whole labeled set $D$ (red line below).

On the other hand, we want to have a large enough validation split, such that the validation empirical risk is a good estimate of the population risk of a prediction function. This happens if $K$ is too large (blue line below).

We assume the following notation:

- Population risk: $R(f) := \mathbb{E}_{x, y} \left[ \left(y - f(x)\right)^2 \right]$
- Validation risk on the $k^{\text{th}}$ validation split: $\hat{R}^{(k)}(f) := \frac{1}{\left| D_V^{(k)} \right|} \sum_{(x_i, y_i)\in D_V^{(k)}} \left(y_i - f(x_i)\right)^2$
- Linear regression estimator trained on the whole labeled set: $\hat{f}_{full} := \text{argmin}_f \frac{1}{\left| D \right|} \sum_{(x_i, y_i)\in D} \left(y_i - f(x_i)\right)^2$
- Linear regression estimator trained on the $k^{\text{th}}$ training split: $\hat{f}_{CV}^{(k)} := \text{argmin}_f \frac{1}{\left| D_T^{(k)} \right|} \sum_{(x_i, y_i)\in D_T^{(k)}} \left(y_i - f(x_i)\right)^2$


- Average population risk of CV estimators: $R_{CV} := \frac{1}{K} \sum_{k=1}^K R\left( \hat{f}_{CV}^{(k)}(x)\right)$
- Empirical CV risk computed on the validation set: $\hat{R}_{CV}:= \frac{1}{K} \sum_{k=1}^K \hat{R}^{(k)} \left( \hat{f}_{CV}^{(k)}(x) \right)$
- Average squared difference between the validation risk and the population risk: $\Delta^2 := \frac{1}{K} \sum_{k=1}^K \left( R\left( \hat{f}_{CV}^{(k)}(x)\right) - \hat{R}^{(k)} \left( \hat{f}_{CV}^{(k)}(x) \right) \right)^2$


In [114]:
num_runs = 1

def get_risks_for_kfold_cv(Ks):
  n = 400
  d = 100
  noise_sigma = 1
  snr = 1

  X, y, beta_star, Sigma = generate_data(n=n, d=d, snr=snr, noise_sigma=noise_sigma, seed=99)

  lin_reg_full_set = LinearRegression(fit_intercept=False)
  lin_reg_full_set.fit(X, y)
  lin_reg_full_population_risk = compute_population_risk(beta_star, lin_reg_full_set.coef_, noise_sigma, Sigma)

  pop_risks, validation_risks, cv_vs_pop_squared_difference = [], [], []
  for K in Ks:
    curr_K_pop_risks, curr_K_cv_risks = [], []
    kf = KFold(n_splits=K)
    for train_index, valid_index in kf.split(X):
      lin_reg = LinearRegression(fit_intercept=False)
      lin_reg.fit(X[train_index], y[train_index])
      curr_K_pop_risks.append(compute_population_risk(beta_star, lin_reg.coef_, noise_sigma, Sigma))
      curr_K_cv_risks.append(compute_empirical_risk(lin_reg.coef_, X[valid_index], y[valid_index]))
    curr_K_pop_risks = np.array(curr_K_pop_risks)
    curr_K_cv_risks = np.array(curr_K_cv_risks)
    pop_risks.append(curr_K_pop_risks.mean())
    validation_risks.append(curr_K_cv_risks.mean())
    cv_vs_pop_squared_difference.append(np.power(curr_K_pop_risks - curr_K_cv_risks, 2).mean())

  pop_risks = np.array(pop_risks)
  validation_risks = np.array(validation_risks)
  return {
      "valid_vs_pop_risk": cv_vs_pop_squared_difference,
      "full_vs_cv_pop_risk": np.fabs(pop_risks - lin_reg_full_population_risk),
      "valid_cv_risk_vs_full_pop_risk": np.fabs(validation_risks - lin_reg_full_population_risk),
  }

Ks = [2, 5, 10, 50, 100]
aggregated_results = repeat_experiment(num_runs, get_risks_for_kfold_cv, {"Ks": Ks})

fig = go.Figure()
# Validation risk vs Population risk.
fig.add_trace(go.Scatter(x=Ks, y=aggregated_results["valid_vs_pop_risk"], line_dash="dash", marker_color="blue",
                         name=r"$\Delta^2$"))
# Full set estimator vs CV estimator (pop. risk).
fig.add_trace(go.Scatter(x=Ks, y=aggregated_results["full_vs_cv_pop_risk"], line_dash="dot", marker_color="red",
                         name=r"$\left|R_{CV}\left(\hat{f}_{CV}\right) - R\left(\hat{f}_{full}\right)\right|$"))
# Validation risk of CV estimator vs Population risk of full set estimator.
fig.add_trace(go.Scatter(x=Ks, y=aggregated_results["valid_cv_risk_vs_full_pop_risk"], marker_color="black", line_width=3, 
                         name=r"$\left|\hat{R}_{CV}\left(\hat{f}_{CV}\right) - R\left(\hat{f}_{full}\right)\right|$"))

fig.update_layout(
  height=figure_height,
  width=figure_width,
  yaxis_title="Risk difference",
  xaxis_title="Number of folds",
)
fig.show()

# Hyperparameter tuning using CV

The figures below illustrate the aforementioned trade-off for a concrete use case: tuning the ridge coefficient of a linear regression problem.

We will compare the best value of the ridge coefficient obtained using the population risk, i.e. $\lambda^*$ with the best value obtained using $K$-fold cross-validation, i.e. $\hat{\lambda}$, where we vary the number of folds $K$.

To make it easier to see the difference in the best $\lambda$ between the different settings, we shift the risk for all curves by $R(\lambda^*) - \hat{R}_{CV}(\hat{\lambda})$, where $\hat{R}_{CV}(\hat{\lambda})$ is the minimum value attained by the empirical/cross-validation risk wrt $\lambda$ and $R(\lambda^*)$ is the minimum value of the population risk wrt $\lambda$. We can perform this transformation because it doesn't alter the ranking of the $\lambda$ values (i.e. the best $\lambda$ value will still be the optimal value after the transformation).

In [150]:
n = 100
d = 10
all_noise_sigmas = [0, 0.1, 0.5, 1]
snr = 1

def plot_risk_for_different_ridge_coefficients(K, noise_sigma):
  K = int(K)
  X, y, beta_star, Sigma = generate_data(n=n, d=d, snr=snr, noise_sigma=noise_sigma, seed=21)
  
  all_ridge_coefficients = np.arange(0, 21, 1)
  cv_risks = []
  full_estimator_population_risk = []
  for ridge_coef in all_ridge_coefficients:
    ridge_reg = Ridge(alpha=ridge_coef, fit_intercept=False)
    scores = cross_val_score(ridge_reg, X, y, scoring="neg_mean_squared_error", cv=K)
    cv_risks.append(-scores.mean())

    full_ridge_reg = Ridge(alpha=ridge_coef, fit_intercept=False)
    full_ridge_reg.fit(X, y)
    full_estimator_population_risk.append(compute_population_risk(beta_star, full_ridge_reg.coef_, noise_sigma, Sigma))
  
  fig = go.Figure()
  # Shift the CV line by a constant amount for illustration purposes. Note that this transformation 
  # does not change the ranking of the ridge coefficient values.
  best_population_risk = np.min(full_estimator_population_risk)
  best_empirical_risk = np.min(cv_risks)
  cv_risks = cv_risks + (best_population_risk - best_empirical_risk)
  fig.add_trace(go.Scatter(x=all_ridge_coefficients, y=cv_risks, 
                           marker_color="blue", name=f"Cross validation estimator w/ K={K}"))  
  fig.add_trace(go.Scatter(x=all_ridge_coefficients, y=full_estimator_population_risk, 
                           marker_color="red", name=f"Pop. risk of full estimator"))

  if noise_sigma == 0.5:
    yaxis_range = [0.26, 0.34]
  elif noise_sigma == 1:
    yaxis_range = [1.08, 1.15]
  elif noise_sigma == 0.1:
    yaxis_range = [0.01, 0.02]
  else:
    yaxis_range = [0, 0.02]
  
  best_empirical_ridge_coef = all_ridge_coefficients[np.argmin(cv_risks)]
  best_population_ridge_coef = all_ridge_coefficients[np.argmin(full_estimator_population_risk)]
  fig.add_vline(x=best_empirical_ridge_coef, line_dash="dot", line_color="blue")
  fig.add_vline(x=best_population_ridge_coef, line_dash="dot", line_color="red")
  fig.add_annotation(
      yref='paper',
      x=best_empirical_ridge_coef-0.4, y=1.149,
      text="$\huge\hat{\lambda}$",
      showarrow=False)
  fig.add_annotation(
      yref='paper',
      x=best_population_ridge_coef+0.4, y=1.13,
      text="$\huge \lambda^*$",
      showarrow=False)

  fig.update_layout(
    yaxis_range=yaxis_range,
    height=figure_height,
    width=figure_width,
    yaxis_title="Shifted risk",
    xaxis_title="Ridge coefficient",
    hovermode='x'
  )
  fig.show()

interact(plot_risk_for_different_ridge_coefficients,
         K=ipywidgets.FloatSlider(value=2,
                                  min=2,
                                  max=10,
                                  step=1,
                                  readout_format='d',
                                  description='Number of folds:',
                                  style={'description_width': 'initial'},
                                  continuous_update=False),
         noise_sigma=ipywidgets.Dropdown(options=all_noise_sigmas,
                                         value=0.5,
                                         description='Noise level:',
                                         disabled=False,
                                         style={'description_width': 'initial'},
                                         continuous_update=True),);

interactive(children=(FloatSlider(value=2.0, continuous_update=False, description='Number of folds:', max=10.0…

### Impact of the number of folds

In [157]:
n = 100
d = 10
all_noise_sigmas = [0, 0.1, 0.5, 1]
snr = 1

def plot_risk_for_different_ridge_coefficients_and_Ks(noise_sigma):
  X, y, beta_star, Sigma = generate_data(n=n, d=d, snr=snr, noise_sigma=noise_sigma, seed=21)
  Ks = [2, 5, 10]
  
  fig = go.Figure()
  all_ridge_coefficients = np.arange(0, 30, 1)
  full_estimator_population_risk = []
  
  for ridge_coef in all_ridge_coefficients:
    full_ridge_reg = Ridge(alpha=ridge_coef, fit_intercept=False)
    full_ridge_reg.fit(X, y)
    full_estimator_population_risk.append(compute_population_risk(beta_star, full_ridge_reg.coef_, noise_sigma, Sigma))
  full_estimator_population_risk = np.array(full_estimator_population_risk)
  best_population_risk = np.min(full_estimator_population_risk)
  fig.add_trace(go.Scatter(x=all_ridge_coefficients, y=full_estimator_population_risk, 
                           marker_color="red", name=f"Pop. risk of full estimator"))

  for i, K in enumerate(Ks):
    cv_risks = []
    for ridge_coef in all_ridge_coefficients:
      ridge_reg = Ridge(alpha=ridge_coef, fit_intercept=False)
      scores = cross_val_score(ridge_reg, X, y, scoring="neg_mean_squared_error", cv=K)
      cv_risks.append(-scores.mean())
    
    cv_risks = np.array(cv_risks)
    best_empirical_risk = np.min(cv_risks)
    
    # Shift all lines by a constant amount for illustration purposes. Note that this transformation 
    # does not change the ranking of the ridge coefficient values.
    cv_risks = cv_risks + (best_population_risk - best_empirical_risk)
    fig.add_trace(go.Scatter(x=all_ridge_coefficients, y=cv_risks, marker_color=colors[i],
                             name=f"Cross validation estimator w/ K={K}"))  

  if noise_sigma == 0.5:
    yaxis_range = [0.272, 0.285]
  elif noise_sigma == 1:
    yaxis_range = [1.07, 1.15]
  elif noise_sigma == 0.1:
    yaxis_range = [0.01, 0.02]
  else:
    yaxis_range = [0, 0.01]

  fig.update_layout(
    yaxis_range=yaxis_range,
    height=figure_height,
    width=figure_width,
    yaxis_title="Shifted risk",
    xaxis_title="Ridge coefficient",
    hovermode='x'
  )
  fig.show()

interact(plot_risk_for_different_ridge_coefficients_and_Ks,
         noise_sigma=ipywidgets.Dropdown(options=all_noise_sigmas,
                                         value=1,
                                         description='Noise level:',
                                         disabled=False,
                                         style={'description_width': 'initial'},
                                         continuous_update=True),);

interactive(children=(Dropdown(description='Noise level:', index=3, options=(0, 0.1, 0.5, 1), style=Descriptio…

We can also see that the empirical risk minimizer converges to the population minimizer, by comparing the difference between the best ridge coefficient $\lambda$ obtained through $K$-fold cross-validation, and the best value obtained using the whole labeled set for training and the population risk for hyperparameter tuning.

In [129]:
n = 100
d = 10
snr = 1
all_noise_sigmas = [0, 0.75, 1]
all_Ks = np.arange(2, 11)

deviation_from_best_lambdas = {}
all_ridge_coefficients = np.arange(0, 20, 1)

for noise_sigma in all_noise_sigmas:
  X, y, beta_star, Sigma = generate_data(n=n, d=d, snr=snr, noise_sigma=noise_sigma, seed=21)
  deviation_from_best_lambdas[noise_sigma] = []
  for K in all_Ks:
    cv_risks, full_estimator_population_risk = [], []
    for ridge_coef in all_ridge_coefficients:
      ridge_reg = Ridge(alpha=ridge_coef, fit_intercept=False)
      scores = cross_val_score(ridge_reg, X, y, scoring="neg_mean_squared_error", cv=K)
      cv_risks.append(-scores.mean())

      full_ridge_reg = Ridge(alpha=ridge_coef, fit_intercept=False).fit(X, y)
      full_estimator_population_risk.append(compute_population_risk(beta_star, full_ridge_reg.coef_, noise_sigma, Sigma))

    best_empirical_lambda = all_ridge_coefficients[np.argmin(cv_risks)]
    best_population_lambda = all_ridge_coefficients[np.argmin(full_estimator_population_risk)]
    deviation_from_best_lambdas[noise_sigma].append(np.fabs(best_empirical_lambda - best_population_lambda))

fig = go.Figure()
for i, noise_sigma in enumerate(all_noise_sigmas):
  fig.add_trace(go.Scatter(x=all_Ks, y=deviation_from_best_lambdas[noise_sigma], marker_color=colors[i],
                           name=f"Noise level {noise_sigma}"))

fig.update_layout(
  height=figure_height,
  width=figure_width,
  yaxis_title=r"$\text{Deviation from }\lambda^*$",
  xaxis_title="Number of folds",
  hovermode='x'
)
fig.show()